#### Measurement Framework Library
# MFLib API Examples: Download Grafana Graphs as PNGs

This notebook shows how to use MFVis to download Grafana graphs as PNG files.

# Prerequisites
The slice has to have already been instrumentized. See the [Instrumentize a Slice](./instrumentize_a_slice.ipynb) notebook.

### General Imports

In [ ]:
import os
import json
import traceback

### Set slice information
MFVis uses the slice name to find the slice.

In [ ]:
slice_name = "MyMonitoredSlice"

## Import mfvis and create new mfvis object with slice name

In [ ]:
# If fabrictestbed_extensions are installed you can use:
from fabrictestbed_extensions.mflib.mfvis import mfvis
# If you are using a local copy of the mflib.py file use:
#from mfvis import mfvis

Create the new MFVis object by passing in the name of the slice from which you will be getting graphs.
This will take a few seconds. 
Note: If the slice has not been previously initialized and instrumentized then this step will attempt to initialize the slice. That will take ~ 5 minutes. However, as of this writing, if the slice is not instrumentized, instrumentize will not be performed.

In [ ]:
mfv = mfvis(slice_name)
# Or, optionally, add a specific directory where the downloaded information will be stored. By default this is in the /tmp/mflib directory. 
#  Change it to a relative directory to this notebook. Then, in JupyterHub, right click on this notebook tab and click "Show in File Browser" to easily access the newly created directory.
#mfv = mfvis(slice_name, local_storage_directory="persistent_local_mflib_storage")


## Download measurement graphs as .png files
Specifing the graph you want to download requires several items.

1) The arguments that must be specified include dashboard_name, panel_name, time_filter and node_name.
1) Whether the interface_name should be specified is based on dashboard_name. Currently, only 'network-traffic-dashboard' requires interface_name.
1) The save_as_filename is optional. The save_as_argument allows the users to define the name of the downloaded .png file. The default file will be in the storage directory of the slice. In either case, the saved filename will be returned.
1) The timezone argument allows the users to view and download the graph image using timeseries in their time zone. Default graph time is UTC 

There are several helper methods to determine the available graphs.

### Get available information

In [ ]:
print ("Available dashboards: "+str(mfv.get_dashboard_names()))
print ("Available time filters: "+ str(mfv.get_available_time_filter_names()))
print ("Available nodes: "+ str(mfv.get_available_node_names()))

#### Get available panel names based on the dashboard name

In [ ]:
dashboard='network-traffic-dashboard'
print (mfv.get_panel_names(dashboard))

#### Get available interface names based on node name

In [ ]:
node='Node1'
print (mfv.get_interface_names(node)[0])

### Example: Specify arguments and download graphs in 'node-exporter-full' dashboard

No filename specified.

In [ ]:
dashboard1= 'node-exporter-full'
panel1='CPU Basic'
timefilter1= 'Last 5 minutes'
node1='Node1'

# Call the API to download the file. 
# When file name is not specified, the default is /tmp/mflib/<slice_name>/grafana_manager/rendered/{panel_name}.png
#  If you specified a local_storage_directory earlier, then the image will be in that directory under <slice_name>/grafana_manager/rendered/{panel_name}.png
saved_filename =  mfv.download_graph(dashboard1, panel1, timefilter1, node1) 
print(saved_filename)

Relative filename specified.

In [ ]:
save_as_filename = "relative_filename.png"
# Call the API to download the file. 
# When using a relative file path, the file will be saved in the same directory as this notebook. 
#   Then, in JupyterHub, right click on this notebook tab and click "Show in File Browser" to easily access the newly file.
saved_filename =  mfv.download_graph(dashboard1, panel1, timefilter1, node1, save_as_filename=filepath) 
print(saved_filename)

Full path filename specified.

In [ ]:
save_as_filename = "/home/fabric/work/full_path_test.png"
# Call the API to download the file. 
# When using a full file path, the file will be saved there. 
saved_filename =  mfv.download_graph(dashboard1, panel1, timefilter1, node1, save_as_filename=save_as_filename) 
print(saved_filename)

### Example: Specify arguments and download graphs in 'network-traffic-dashboard'

In [ ]:
dashboard2='network-traffic-dashboard'
panel2='Network Traffic by Packets'
timefilter2= 'Last 15 minutes'
node2='Node2'

# See available interface names for node2
print(mfv.get_interface_names(node2)[0])

In [ ]:
interface='ens8'
saved_filename = mfv.download_graph(dashboard2, panel2, timefilter2, node2,interface_name=interface)
print(saved_filename)

## Setting the Graph's Time Zone 
All FABRIC racks are set to user UTC time.   
If you do not pass the time_zone argument the all graphs will default to UTC.

If you would prefer to have a graph labeled with a certain time zone you may add the optional time_zone argument.  
There are 3 options for getting the needed time zone string.

#### Option 1
If you know your timezone offset you can just use GMT+/-offset.
So, depending on daylight savings time, US Eastern time zone would be GMT-5 (winter) or GMT-4 (summer),
US Central time zone wouldb be GMT-6 (winter) or GMT-5 (summer), etc..
France would be GMT+1 (winter) GMT+2 (summer).    

##### Option 2 
Find the timezone of your browser.

In [ ]:
from IPython.display import Javascript
Javascript("element.append(Intl.DateTimeFormat().resolvedOptions().timeZone);")

##### Option 3
Find the desired time zone from a list of time zones.

In [ ]:
import pytz

tzs = pytz.all_timezones
print(tzs)


In [ ]:
# List the possible regions
regions = set([z.split("/")[0] for z in tzs ])
print(sorted(regions))

In [ ]:
# Choose a region
region = "America"

# List possible subregions
subregions = set([z.split("/")[1] for z in tzs if f"{region}" in z ])
print(sorted(subregions))

In [ ]:

# Choose the subregion
subregion = "Chicago"

# Get the timezone string by region & subregion
time_zone = [z for z in tzs if f"{region}/{subregion}" in z][0]

print(time_zone)

### Add the time zone to the download graph method.

In [ ]:
# Specify the optional arguments
save_as_filename2 = 'new_york_timezone.png'
time_zone='America/New_York'
saved_filename =  mfv.download_graph(dashboard2, panel2, timefilter2, node2, save_as_filename=save_as_filename2, interface_name=interface, time_zone=time_zone)
print(saved_filename)

# Get graph url
You can get the URL for a graph to use anytime while the slice is still up and running.
Note that the SSH Tunnel will need to be active when using the URL. See below cell Open SSH Tunnel.

In [ ]:
dashboard2='network-traffic-dashboard'
panel2='Network Traffic by Packets'
timefilter2= 'Last 15 minutes'
node2='Node2'

# See available interface names for node2
print(mfv.get_interface_names(node2)[0])


In [ ]:
interface='ens8'

In [ ]:
print(mfv.render_graph_url(dashboard2, panel2, timefilter2, node2,interface_name=interface))

You may optionally add the time zone.

In [ ]:
print(mfv.render_graph_url(dashboard2, panel2, timefilter2, node2,interface_name=interface, time_zone=time_zone))

## Open SSH Tunnel
Ensure that the tunnel is open to the Measurement Node running Grafana.
If you are running multiple slices you may want to set a custom local port for your slice. You may choose any port you want being sure to avoid common ports.

The first time you click on the render_graph_url's you will have to accept the self-signed certificate.  
Click on the "Continue to localhost (unsafe)" link.
This is required because the web server setup is dynamically created in your slice so obtaining a signed certificate is impractical.

In [ ]:
#mfv.grafana_tunnel_local_port = 10009
mfv.grafana_tunnel